# IR Practical No.-3
                           

Write a program Write a program to construct a Bayesian network considering medical data. Use this model to demonstrate the diagnosis of heart patients using the standard Heart Disease Data Set.

In [1]:
!pip install statsmodels==0.13.5

  Using cached statsmodels-0.13.5.tar.gz (18.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [87 lines of output]
  <string>:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  
  Error compiling Cython file:
  ------------------------------------------------------------
  ...
  cimport cython
  cimport numpy as np
  import numpy as np
  
  ctypedef np.float64_t DOUBLE
  ctypedef np.int_t INT
          ^
  ------------------------------------------------------------
  
  statsmodels\nonparametric\linbin.pyx:13:9: 'int_t' is not a type identifier
  Compiling statsmodels/tsa/_stl.pyx because it depends on C:\Users\Hitesh\AppData\Local\Temp\pip-build-env-fretly6f\overlay\Lib\site-packages\Cython\Includes\libc\math.pxd.
  Compiling statsmodels/tsa/holtwinters/_expon

In [2]:
import pandas as pd
import numpy as np
try:
    from pgmpy.models import DiscreteBayesianNetwork as BN   # for pgmpy ≥ 1.0.0
except ImportError:
    from pgmpy.models import BayesianModel as BN             # fallback for older versions
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination


ImportError: cannot import name '_lazywhere' from 'scipy._lib._util' (C:\Users\Hitesh\AppData\Roaming\Python\Python313\site-packages\scipy\_lib\_util.py)

In [ ]:
# Load the dataset (keep heart.csv in the same folder)
df = pd.read_csv("heart.csv").replace('?', np.nan).dropna()

# Rename target column if needed
for t in ['target', 'num']:
    if t in df.columns and 'heartdisease' not in df.columns:
        df = df.rename(columns={t: 'heartdisease'})

# Keep only important columns
cols = ['age','sex','trestbps','fbs','exang','restecg','thalach','chol','heartdisease']
df = df[[c for c in cols if c in df.columns]].copy()

df.head()


In [ ]:
if 'age' in df:       df['age'] = (df['age'].astype(float) > 50).astype(int)
if 'trestbps' in df:  df['trestbps'] = (df['trestbps'].astype(float) > 130).astype(int)
if 'thalach' in df:   df['thalach'] = (df['thalach'].astype(float) < 150).astype(int)  # 1 = low
if 'chol' in df:      df['chol'] = (df['chol'].astype(float) > 200).astype(int)
df = df.dropna().astype(int)

df.head()


In [ ]:
features = [c for c in df.columns if c != 'heartdisease']
model = BN([('heartdisease', f) for f in features])

# Learn CPDs
model.fit(df, estimator=MaximumLikelihoodEstimator)
infer = VariableElimination(model)

print("Learning CPDs using Maximum Likelihood Estimators...\n")
print("Inferencing with Bayesian Network:")


In [ ]:
def bin_evidence(evidence):
    b = {}
    for k, v in evidence.items():
        if k == 'age': b[k] = int(float(v) > 50)
        elif k == 'trestbps': b[k] = int(float(v) > 130)
        elif k == 'chol': b[k] = int(float(v) > 200)
        elif k == 'thalach': b[k] = int(float(v) < 150)
        else: b[k] = int(v)
    return b


In [ ]:
print("\n1. Probability of HeartDisease given Age = 28")
print(infer.query(['heartdisease'], evidence=bin_evidence({'age': 28})))

print("\n2. Probability of HeartDisease given Cholesterol = 100")
print(infer.query(['heartdisease'], evidence=bin_evidence({'chol': 100})))
